In [1]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_column', 500)

# LCFF and district shapefiles

In [2]:
# import and clean lcff data
lcff = pd.read_csv('lcff_data/lcffsnapshot18an.csv')
lcff.columns = lcff.columns.str.strip()
# strip leading/trailing spaces
lcff.drop(lcff.tail(1).index,inplace=True)
# extracting state totals, removing from df
# Slicing by district level cds code
lcff['County Code'] = lcff['County Code'].astype(str)
lcff['District Code'] = lcff['District Code'].astype(int)
lcff['District Code'] = lcff['District Code'].astype(str)
lcff['School Code'] = lcff['School Code'].astype(int)
lcff['School Code'] = lcff['School Code'].astype(str)
# Changing to strings
lcff['School Code'] = lcff['School Code'].apply(lambda x: '{0:0>7}'.format(x))
# Front fill school codes to == 7
lcff = lcff.loc[lcff['School Code'] == '0000000']
# select districts, district entries do not have a school level cds code
lcff['cds'] = lcff['County Code'] + lcff['District Code'] + lcff['School Code']
# Combining into cds code
col_name='cds'
first_col = lcff.pop(col_name)
lcff.insert(0, col_name, first_col)
# Moving to front 
lcff.drop('County Code', axis=1, inplace=True)
# lcff.drop('District Code', axis=1, inplace=True)
lcff.drop('School Code', axis=1, inplace=True)
lcff = lcff.rename(columns={'Charter Number': 'charter_number',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_percentage',
                     'Base Grant Funding\nTarget, C-5': 'base_grant',
                     'Supplemental Grant Funding\nTarget, D-7': 'supplemental_grant',
                     'Concentration Grant Funding\nTarget, E-8': 'concentration_grant',
                     'Total LCFF Target Entitlement\nTarget, G-1/F-1': 'total_grants',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_count',
                     'Necessary Small Schools Allowance\nTarget, F-1': 'nec_small_schools',
                     'Local Educational Agency' : 'districtname'      
                           })
lcff = lcff[[  'districtname',
               'unduplicated_pupil_count', 
               'base_grant', 
               'supplemental_grant', 
               'concentration_grant', 
               'total_grants', 
               'cds' ]].copy()

In [3]:
lcff

,districtname,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds
10,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
16,Albany City Unified,0.2833,"29,644,972","1,679,684",-,"31,352,540",01611270000000
17,Berkeley Unified,0.3519,"78,556,532","5,528,808",-,"89,369,621",01611430000000
19,Castro Valley Unified,0.2904,"74,911,585","4,350,864",-,"79,554,607",01611500000000
20,Emery Unified,0.7953,"5,702,229","906,997","699,378","7,326,875",01611680000000
...,...,...,...,...,...,...,...
2233,Camptonville Elementary,0.6988,"420,692","58,796","31,299","537,244",58727280000000
2235,Marysville Joint Unified,0.8168,"76,193,862","12,447,028","10,164,261","100,261,913",58727360000000
2238,Plumas Lake Elementary,0.3868,"10,261,440","793,825",-,"11,108,901",58727440000000
2239,Wheatland,0.4783,"9,947,133","951,543",-,"11,234,349",58727510000000


In [4]:
new = pd.read_csv('California_School_District_Areas_2018-19.csv')
new = new.rename(columns={'DistrictName' : 'districtname'})

In [5]:
new

,FID,Year,FedID,CDCode,CDSCode,CountyName,districtname,DistrictType,GradeLow,GradeHigh,GradeLowCensus,GradeHighCensus,AssistStatus,CongressUS,SenateCA,AssemblyCA,LocaleDistrict,UpdateNotes,EnrollTotal,EnrollCharter,EnrollNonCharter,AAcount,AApct,AIcount,AIpct,AScount,ASpct,FIcount,FIpct,HIcount,HIpct,MRcount,MRpct,PIcount,PIpct,WHcount,WHpct,NRcount,NRpct,ELcount,ELpct,FOScount,FOSpct,HOMcount,HOMpct,MIGcount,MIGpct,SWDcount,SWDpct,SEDcount,SEDpct,UPcount,UPpct,ELATested,ELAStdMetPct,MathTested,MathStdMetPct,CCPrepCohortCount,CCPrepPct,AbsentEligCount,AbsentPct,GradCohortCount,GradPct,UCCSUReqMetPct,DropOutPct,EnrollCumulative,SuspPct,SHAPE_Length,SHAPE_Area
0,1,2018-19,601770,161119,1611190000000,Alameda,Alameda Unified,Unified,KG,12,KG,12,Differentiated Assistance,13,09,18,"21 - Suburban, Large",2018-19 District Boundary Adjustment/Correction,11260,1877,9383,825,7.300000,28,0.2,2854,25.299999,683,6.1,1960,17.400000,1310,11.6,67,0.6,3244,28.799999,289,2.6,1601,14.2,26,0.2,67,0.6,0,0.0,1289,11.4,3523,31.299999,3144,33.500000,4567.0,69.400002,4587.0,59.700001,773.0,56.799999,11453.0,9.4,882.0,88.900002,58.900002,5.2,11600.0,2.6,57196.403944,4.733304e+07
1,2,2018-19,601860,161127,1611270000000,Alameda,Albany City Unified,Unified,KG,12,KG,12,General Assistance,13,09,15,"21 - Suburban, Large",2018-19 District Boundary Adjustment/Correction,3682,0,3682,111,3.000000,4,0.1,1063,28.900000,47,1.3,592,16.100000,510,13.9,6,0.2,1163,31.600000,186,5.1,583,15.8,2,0.1,12,0.3,0,0.0,312,8.5,777,21.100000,1118,30.400000,1773.0,79.199997,1795.0,72.000000,302.0,62.599998,3763.0,6.4,301.0,94.699997,63.900002,1.7,3830.0,1.9,12870.772156,7.161276e+06
2,3,2018-19,604740,161143,1611430000000,Alameda,Berkeley Unified,Unified,KG,12,KG,12,Differentiated Assistance,13,09,15,"12 - City, Midsize",2018-19 District Boundary Adjustment/Correction,10194,383,9811,1440,14.100000,16,0.2,731,7.200000,83,0.8,2438,23.900000,1465,14.4,14,0.1,3999,39.200001,8,0.1,953,9.3,23,0.2,265,2.6,0,0.0,1148,11.3,3331,32.700001,3275,33.400002,4614.0,66.699997,4549.0,61.700001,790.0,63.299999,10404.0,9.7,856.0,87.400002,83.300003,10.7,10534.0,1.4,44426.287405,4.376389e+07
3,4,2018-19,607800,161150,1611500000000,Alameda,Castro Valley Unified,Unified,KG,12,KG,12,General Assistance,"13,15","07,09,10","18,20","21 - Suburban, Large",2018-19 District Boundary Adjustment/Correction,9269,0,9269,489,5.300000,19,0.2,2539,27.400000,420,4.5,2204,23.799999,909,9.8,47,0.5,2480,26.799999,162,1.7,1021,11.0,12,0.1,68,0.7,0,0.0,800,8.6,2284,24.600000,2739,29.600000,4844.0,68.099998,4850.0,63.200001,737.0,57.099998,9433.0,7.4,742.0,92.599998,55.900002,3.0,9532.0,2.2,142338.494091,2.833205e+08
4,5,2018-19,612630,161168,1611680000000,Alameda,Emery Unified,Unified,KG,12,KG,12,Differentiated Assistance,13,09,15,"21 - Suburban, Large",2018-19 District Boundary Adjustment/Correction,743,0,743,326,43.900002,1,0.1,83,11.200000,11,1.5,185,24.900000,62,8.3,8,1.1,67,9.000000,0,0.0,160,21.5,1,0.1,7,0.9,0,0.0,55,7.4,600,80.800003,613,83.099998,376.0,37.099998,377.0,23.299999,40.0,22.500000,760.0,20.5,40.0,100.000000,40.000000,0.0,780.0,3.6,13762.869292,5.368212e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,940,2018-19,607260,5872728,58727280000000,Yuba,Camptonville Elementary,Elementary,KG,12,KG,8,Differentiated Assistance,03,04,03,"22 - Suburban, Midsize",NaN,481,433,48,6,1.200000,8,1.7,2,0.400000,1,0.2,84,17.500000,51,10.6,3,0.6,320,66.500000,6,1.2,7,1.5,3,0.6,1,0.2,0,0.0,71,14.8,263,54.700001,39,81.199997,32.0,68.800003,32.0,68.800003,NaN,NaN,533.0,5.4,44.0,95.500000,16.700001,4.5,552.0,1.4,79862.488656,2.462935e+08
940,941,2018-19,624090,5872736,58727360000000,Yuba,Marysville Joint Unified,Unified,KG,12,KG,12,Differentiated Assistance,"01,03",04,"01,03","22 - Suburban, Mi

In [6]:
k = pd.merge(lcff, new, on=['districtname'], how='left')
k

,districtname,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds,FID,Year,FedID,CDCode,CDSCode,CountyName,DistrictType,GradeLow,GradeHigh,GradeLowCensus,GradeHighCensus,AssistStatus,CongressUS,SenateCA,AssemblyCA,LocaleDistrict,UpdateNotes,EnrollTotal,EnrollCharter,EnrollNonCharter,AAcount,AApct,AIcount,AIpct,AScount,ASpct,FIcount,FIpct,HIcount,HIpct,MRcount,MRpct,PIcount,PIpct,WHcount,WHpct,NRcount,NRpct,ELcount,ELpct,FOScount,FOSpct,HOMcount,HOMpct,MIGcount,MIGpct,SWDcount,SWDpct,SEDcount,SEDpct,UPcount,UPpct,ELATested,ELAStdMetPct,MathTested,MathStdMetPct,CCPrepCohortCount,CCPrepPct,AbsentEligCount,AbsentPct,GradCohortCount,GradPct,UCCSUReqMetPct,DropOutPct,EnrollCumulative,SuspPct,SHAPE_Length,SHAPE_Area
0,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000,1.0,2018-19,601770.0,161119.0,1.611190e+12,Alameda,Unified,KG,12.0,KG,12.0,Differentiated Assistance,13,09,18,"21 - Suburban, Large",2018-19 District Boundary Adjustment/Correction,11260.0,1877.0,9383.0,825.0,7.300000,28.0,0.2,2854.0,25.299999,683.0,6.1,1960.0,17.400000,1310.0,11.6,67.0,0.6,3244.0,28.799999,289.0,2.6,1601.0,14.2,26.0,0.2,67.0,0.6,0.0,0.0,1289.0,11.4,3523.0,31.299999,3144.0,33.500000,4567.0,69.400002,4587.0,59.700001,773.0,56.799999,11453.0,9.4,882.0,88.900002,58.900002,5.2,11600.0,2.6,57196.403944,4.733304e+07
1,Albany City Unified,0.2833,"29,644,972","1,679,684",-,"31,352,540",01611270000000,2.0,2018-19,601860.0,161127.0,1.611270e+12,Alameda,Unified,KG,12.0,KG,12.0,General Assistance,13,09,15,"21 - Suburban, Large",2018-19 District Boundary Adjustment/Correction,3682.0,0.0,3682.0,111.0,3.000000,4.0,0.1,1063.0,28.900000,47.0,1.3,592.0,16.100000,510.0,13.9,6.0,0.2,1163.0,31.600000,186.0,5.1,583.0,15.8,2.0,0.1,12.0,0.3,0.0,0.0,312.0,8.5,777.0,21.100000,1118.0,30.400000,1773.0,79.199997,1795.0,72.000000,302.0,62.599998,3763.0,6.4,301.0,94.699997,63.900002,1.7,3830.0,1.9,12870.772156,7.161276e+06
2,Berkeley Unified,0.3519,"78,556,532","5,528,808",-,"89,369,621",01611430000000,3.0,2018-19,604740.0,161143.0,1.611430e+12,Alameda,Unified,KG,12.0,KG,12.0,Differentiated Assistance,13,09,15,"12 - City, Midsize",2018-19 District Boundary Adjustment/Correction,10194.0,383.0,9811.0,1440.0,14.100000,16.0,0.2,731.0,7.200000,83.0,0.8,2438.0,23.900000,1465.0,14.4,14.0,0.1,3999.0,39.200001,8.0,0.1,953.0,9.3,23.0,0.2,265.0,2.6,0.0,0.0,1148.0,11.3,3331.0,32.700001,3275.0,33.400002,4614.0,66.699997,4549.0,61.700001,790.0,63.299999,10404.0,9.7,856.0,87.400002,83.300003,10.7,10534.0,1.4,44426.287405,4.376389e+07
3,Castro Valley Unified,0.2904,"74,911,585","4,350,864",-,"79,554,607",01611500000000,4.0,2018-19,607800.0,161150.0,1.611500e+12,Alameda,Unified,KG,12.0,KG,12.0,General Assistance,"13,15","07,09,10","18,20","21 - Suburban, Large",2018-19 District Boundary Adjustment/Correction,9269.0,0.0,9269.0,489.0,5.300000,19.0,0.2,2539.0,27.400000,420.0,4.5,2204.0,23.799999,909.0,9.8,47.0,0.5,2480.0,26.799999,162.0,1.7,1021.0,11.0,12.0,0.1,68.0,0.7,0.0,0.0,800.0,8.6,2284.0,24.600000,2739.0,29.600000,4844.0,68.099998,4850.0,63.200001,737.0,57.099998,9433.0,7.4,742.0,92.599998,55.900002,3.0,9532.0,2.2,142338.494091,2.833205e+08
4,Emery Unified,0.7953,"5,702,229","906,997","699,378","7,326,875",01611680000000,5.0,2018-19,612630.0,161168.0,1.611680e+12,Alameda,Unified,KG,12.0,KG,12.0,Differentiated Assistance,13,09,15,"21 - Suburban, Large",2018-19 District Boundary Adjustment/Correction,743.0,0.0,743.0,326.0,43.900002,1.0,0.1,83.0,11.200000,11.0,1.5,185.0,24.900000,62.0,8.3,8.0,1.1,67.0,9.000000,0.0,0.0,160.0,21.5,1.0,0.1,7.0,0.9,0.0,0.0,55.0,7.4,600.0,80.800003,613.0,83.099998,376.0,37.099998,377.0,23.299999,40.0,22.500000,760.0,20.5,40.0,100.000000,40.000000,0.0,780.0,3.6,13762.869292,5.368212e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [7]:
k.LocaleDistrict.unique()
# is this useful?

array(['21 - Suburban, Large', '12 - City, Midsize ',
       '23 - Suburban, Small', '41 - Rural, Fringe', '11 - City, Large',
       '13 - City, Small', '42 - Rural, Distant', '32 - Town, Distant',
       '31 - Town, Fringe', '43 - Rural, Remote', '33 - Town, Remote',
       '22 - Suburban, Midsize', nan], dtype=object)

In [8]:
k.drop('AssistStatus', axis=1, inplace=True)
k.drop('UpdateNotes', axis=1, inplace=True)
k.drop('MRcount', axis=1, inplace=True)
k.drop('MRpct', axis=1, inplace=True)
k.drop('PIcount', axis=1, inplace=True)
k.drop('PIpct', axis=1, inplace=True)
k.drop('NRcount', axis=1, inplace=True)
k.drop('NRpct', axis=1, inplace=True)
k.drop('cds', axis=1, inplace=True)
k = k.rename(columns={'DistrictName' : 'districtname',
                  'AAcount': 'african_american',
                  'AApct' : 'african_american_percent',
                  'AIcount' : 'american_indian',
                  'AIpct' : 'american_indian_percent',
                  'AScount' : 'asian',
                  'ASpct' : 'asian_percent',
                  'FIcount' : 'filipino',
                  'FIpct' : 'filipino_percent',
                  'HIcount' : 'hispanic',
                  'HIpct' : 'hispanic_percent',
                  'WHcount' : 'white',
                  'WHpct' : 'white_percent',
                  'ELcount' : 'english_learner',
                  'ELpct' : 'english_learner_percent',
                  'FOScount' : 'foster',
                  'FOSpct' : 'foster_percent',
                  'HOMcount' : 'homeless',
                  'HOMpct' : 'homeless_percent',
                  'MIGcount' : 'migrant',
                  'MIGpct' : 'migrant_percent',
                  'SWDcount' : 'disabilities',
                  'SWDpct' : 'disabilities_percent',
                                                    })

In [9]:
col_name = 'CountyName'
first_col = k.pop(col_name)
k.insert(0, col_name, first_col)
col_2 = 'EnrollTotal'
second_col = k.pop(col_2)
k.insert(0, col_2, second_col)
# this sucks, is there an easier way to reorder columns?

In [10]:
k.loc[k['CountyName'] == 'Alameda'].sort_values(['EnrollTotal'], ascending=False)

,EnrollTotal,CountyName,districtname,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,FID,Year,FedID,CDCode,CDSCode,DistrictType,GradeLow,GradeHigh,GradeLowCensus,GradeHighCensus,CongressUS,SenateCA,AssemblyCA,LocaleDistrict,EnrollCharter,EnrollNonCharter,african_american,african_american_percent,american_indian,american_indian_percent,asian,asian_percent,filipino,filipino_percent,hispanic,hispanic_percent,white,white_percent,english_learner,english_learner_percent,foster,foster_percent,homeless,homeless_percent,migrant,migrant_percent,disabilities,disabilities_percent,SEDcount,SEDpct,UPcount,UPpct,ELATested,ELAStdMetPct,MathTested,MathStdMetPct,CCPrepCohortCount,CCPrepPct,AbsentEligCount,AbsentPct,GradCohortCount,GradPct,UCCSUReqMetPct,DropOutPct,EnrollCumulative,SuspPct,SHAPE_Length,SHAPE_Area
11,50202.0,Alameda,Oakland Unified,0.7678,"284,749,840","43,726,185","31,009,257","375,304,926",12.0,2018-19,628050.0,161259.0,1.612590e+12,Unified,KG,12.0,KG,12.0,13,09,"15,18","11 - City, Large",13678.0,36524.0,11813.0,23.500000,141.0,0.3,6100.0,12.200000,443.0,0.9,23076.0,46.000000,5175.0,10.300000,15671.0,31.200001,189.0,0.4,902.0,1.8,22.0,0.0,6513.0,13.0,37245.0,74.199997,27799.0,76.199997,17428.0,33.299999,17605.0,27.000000,2701.0,39.500000,51849.0,28.700001,3523.0,76.500000,57.299999,12.6,53118.0,4.4,143476.657365,2.225416e+08
5,35544.0,Alameda,Fremont Unified,0.2780,"284,970,897","15,844,382",-,"305,033,425",6.0,2018-19,614400.0,161176.0,1.611760e+12,Unified,KG,12.0,KG,12.0,"15,17",10,"20,25","21 - Suburban, Large",368.0,35176.0,683.0,1.900000,85.0,0.2,22749.0,64.000000,1823.0,5.1,5149.0,14.500000,3716.0,10.500000,4666.0,13.100000,23.0,0.1,190.0,0.5,69.0,0.2,3333.0,9.4,7332.0,20.600000,9733.0,27.700001,18260.0,78.400002,18312.0,73.900002,2414.0,71.800003,36319.0,5.500000,2484.0,92.900002,55.799999,5.5,36814.0,1.7,164514.790603,3.203380e+08
6,22376.0,Alameda,Hayward Unified,0.7685,"158,763,208","24,401,905","17,344,881","202,238,284",7.0,2018-19,616740.0,161192.0,1.611920e+12,Unified,KG,12.0,KG,12.0,15,10,20,"12 - City, Midsize",2467.0,19909.0,1884.0,8.400000,52.0,0.2,1809.0,8.100000,1570.0,7.0,14444.0,64.599998,1074.0,4.800000,6250.0,27.900000,64.0,0.3,759.0,3.4,389.0,1.7,2395.0,10.7,16285.0,72.800003,15484.0,77.800003,10410.0,36.900002,10545.0,25.400000,1286.0,35.400002,23138.0,14.900000,1585.0,83.900002,52.599998,9.7,23555.0,4.6,87111.969721,1.430745e+08
16,14978.0,Alameda,Pleasanton Unified,0.1601,"121,206,907","3,881,045",-,"125,781,867",17.0,2018-19,600020.0,175101.0,1.751010e+12,Unified,KG,12.0,KG,12.0,15,"07,10","16,20","13 - City, Small",0.0,14978.0,217.0,1.400000,16.0,0.1,6557.0,43.799999,300.0,2.0,1452.0,9.700000,5601.0,37.400002,1706.0,11.400000,9.0,0.1,11.0,0.1,0.0,0.0,1199.0,8.0,1369.0,9.100000,2690.0,18.000000,7944.0,78.900002,7940.0,75.000000,1217.0,76.199997,15331.0,3.400000,1224.0,96.400002,72.800003,1.5,15481.0,2.0,71244.103393,1.670569e+08
7,13738.0,Alameda,Livermore Valley Joint Unified,0.2809,"110,617,503","6,214,491",-,"118,050,908",8.0,2018-19,622110.0,161200.0,1.612000e+12,Unified,KG,12.0,KG,12.0,"11,15",07,"14,16,20","23 - Suburban, Small",0.0,13738.0,202.0,1.500000,37.0,0.3,1216.0,8.900000,422.0,3.1,4252.0,31.000000,6429.0,46.799999,1458.0,10.600000,12.0,0.1,67.0,0.5,192.0,1.4,1795.0,13.1,3529.0,25.700001,3794.0,27.600000,6996.0,62.900002,7005.0,50.200001,1116.0,54.400002,13721.0,5.100000,1126.0,94.099998,48.500000,2.3,13870.0,3.0,206097.407495,1.104156e+09
15,12090.0,Alameda,Dublin Unified,0.1554,"97,373,167","3,026,358",-,"100,522,910",16.0,2018-19,600019.0,175093.0,1.750930e+12,Unified,KG,12.0,KG,12.0,15,"07,10","16,20","21 - Suburban, Large",0.0,12090.0,387.0,3.200000,48.0,0.4,6338.0,52.400002,620.0,5.1,1306.0,10.800000,2384.0,19.700001,918.0,7.600000,6.0,0.0,10.0,0.1,0.0,0.0,810.0,6.7,1094.0,9.000000,1782.0,14.700000,6354.0,78.400002,6356.0,73.400002,682.0,69.099998,12466.0,4.400000,687.0,93.900002,62.599998,3.2,12595.0,1.5,51454.248391,6.939423e+07
10,11281.

Alameda's top districts have a large delta between their unduplicated pupil counts.

In Oakland Unified, the majority (76%) of students are categorized as disadvantaged, compared to only 27% at Fremont Unified. 



# Test scores

In [11]:
mathpop = pd.read_csv("math.csv", index_col=0)
mathpop.columns = mathpop.columns.str.strip()
# strip leading/trailing spaces
engpop = pd.read_csv("ela.csv", index_col=0)
engpop.columns = engpop.columns.str.strip()
# strip leading/trailing spaces
mathpop.shape

(13555, 7)

In [12]:
mathpop.head()

,District Code,districtname,studentgroup,m_currstatus,m_priorstatus,m_change,countyname
163,61119,Alameda Unified,african_american,-83.9,-82.8,-1.1,Alameda
164,61119,Alameda Unified,american_indian,-51.8,-50.0,-1.8,Alameda
165,61119,Alameda Unified,all_students,14.3,10.8,3.4,Alameda
166,61119,Alameda Unified,asian,43.6,44.8,-1.2,Alameda
167,61119,Alameda Unified,english_learners,-13.7,-12.9,-0.9,Alameda


In [13]:
mathpop.loc[mathpop['districtname'] == 'Alameda Unified']
# math scores from the disadvantaged school

,District Code,districtname,studentgroup,m_currstatus,m_priorstatus,m_change,countyname
163,61119,Alameda Unified,african_american,-83.9,-82.8,-1.1,Alameda
164,61119,Alameda Unified,american_indian,-51.8,-50.0,-1.8,Alameda
165,61119,Alameda Unified,all_students,14.3,10.8,3.4,Alameda
166,61119,Alameda Unified,asian,43.6,44.8,-1.2,Alameda
167,61119,Alameda Unified,english_learners,-13.7,-12.9,-0.9,Alameda
168,61119,Alameda Unified,english_learners_only,-42.4,-42.9,0.5,Alameda
169,61119,Alameda Unified,english_only,17.8,13.0,4.7,Alameda
170,61119,Alameda Unified,filipino,-3.5,-12.7,9.3,Alameda
171,61119,Alameda Unified,foster_youth,NaN,NaN,NaN,Alameda
172,61119,Alameda Unified,hispanic,-24.4,-26.7,2.3,Alameda


In [14]:
mathpop.loc[mathpop['districtname'] == 'Fremont Unified']
# math scores from the advantaged school

,District Code,districtname,studentgroup,m_currstatus,m_priorstatus,m_change,countyname
1113,61176,Fremont Unified,african_american,-61.4,-64.3,2.8,Alameda
1114,61176,Fremont Unified,american_indian,4.6,-7.8,12.4,Alameda
1115,61176,Fremont Unified,all_students,61.6,57.4,4.2,Alameda
1116,61176,Fremont Unified,asian,104.5,102.2,2.4,Alameda
1117,61176,Fremont Unified,english_learners,34.5,22.7,11.8,Alameda
1118,61176,Fremont Unified,english_learners_only,-69.0,-51.4,-17.6,Alameda
1119,61176,Fremont Unified,english_only,53.6,49.4,4.2,Alameda
1120,61176,Fremont Unified,filipino,14.8,15.7,-0.8,Alameda
1121,61176,Fremont Unified,foster_youth,NaN,-47.2,NaN,Alameda
1122,61176,Fremont Unified,hispanic,-47.5,-48.5,1.0,Alameda


### RDD
need to query counties to see if they contain school districts that have certain properties. 

breakpoint approach, e.g. one county that countains both: 
* districts with 40-54% upc
* districts with 55-60% upc

or counties that contains both:
* districts with very high (> 80%) upc
* districts with very low (< 20%) upc


hopefully will find multiple counties that meet both criteria to compare.

#### pair treatment/non-treatment by...

geography: neighbors or same county

demographics : upc or census make up

income